# Подготовка к ЛР №2
## Курс "Системы очувствления роботов"
-----

Необходимо реализовать линейный фильтр Калмана.

Задача: Найти положение робота в одномерном пространстве при условии, что мы получаем информацию об изменении положения по данным GPS.<br />

### Исходные данные 
Объект характеризуется положением и скоростью. \
Управление: по ускорению. \
Состояние = 
$$
\mathbf{x}=\left[\begin{array}{c}
p \\
\frac{d p}{d t}=\dot{p}
\end{array}\right]
$$
Вход управления =  $\quad \mathbf{u}=a=\frac{d^{2} p}{d t^{2}}$
\
Модель процесса и модель движения:



$$
\begin{array}{c}
\mathbf{x}_{k}=\left[\begin{array}{cc}
1 & \Delta t \\
0 & 1
\end{array}\right] \mathbf{x}_{k-1}+\left[\begin{array}{c}
0 \\
\Delta t
\end{array}\right] \mathbf{u}_{k-1}+\mathbf{w}_{k-1} \\
y_{k}=\left[\begin{array}{ll}
1 & 0
\end{array}\right] \mathbf{x}_{k}+v_{k} \\
v_{k} \sim \mathscr{N}(0,0.05) \quad \mathbf{w}_{k} \sim \mathscr{N}\left(\mathbf{0},(0.1) \mathbf{1}_{2 \times 2}\right)
\end{array}
$$

## Реализуем фильтрация для одного временного шага

In [4]:
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

Запишем наши исходные данные

In [8]:
x_0 = np.array([[0],[5]])
P_0 = np.array([[0.01 , 0],
                [0, 1]]) #ковариация в моментв ремени 0
w_k = np.array([[0.1 , 0],
                [0, 0.1]]) #ковариация модели движения в момент времени 0 (матрица Q в формулах)
v_k = np.array([0.05]).reshape((1,1)) #ковариация модели измерения в момент времени 0 (в формулах R)
u_0 = -2
y_1 = 2.2
dt = 0.5

Необходимо теперь получить предсказания для x_k и P_k

In [9]:
f_matrix = np.array([[1, dt], [0, 1]])
g_matrix = np.array([[0], [dt]])
x_k = f_matrix.dot(x_0) + g_matrix*u_0 + 0

# propogate covariance
P_k = f_matrix@P_0@f_matrix.T + w_k
P_k

array([[0.36, 0.5 ],
       [0.5 , 1.1 ]])

In [10]:
# теперь надо сделать коррекцию
H = np.array([[1, 0]]) # модель измерения (просто получаем положение)
K_k = P_k@H.T@inv(H@P_k@H.T + v_k)
K_k

array([[0.87804878],
       [1.2195122 ]])

In [11]:
# осталось перерасчитать оценку
x_state = x_k + K_k@(y_1 - H@x_k) + 0
x_state

array([[2.23658537],
       [3.63414634]])

In [18]:
P_state = (np.array([[1, 0],
       [0, 1]]) - K_k@H)@P_k
P_state

array([[0.04390244, 0.06097561],
       [0.06097561, 0.4902439 ]])